# Gaussova eliminacija

## Općenito

Sustav $Ax=b$ se rješava u tri koraka (__bez pivotiranja__)

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija)
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija)
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija)

Ukoliko pivotiramo, tada je

1. $PA=LU$ 
2. $Ly=P^T b$
3. $Ux=y$ 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.694002  0.958648  0.656373  0.864156   0.655747  0.108587
 0.784736  0.59294   0.778007  0.827322   0.298356  0.640561
 0.869287  0.581859  0.138718  0.427552   0.739744  0.70391 
 0.650794  0.810501  0.542283  0.598646   0.780692  0.193462
 0.180298  0.440662  0.261833  0.53353    0.338341  0.356706
 0.416135  0.818816  0.267732  0.0166475  0.914003  0.894401

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
 1.0        0.0        0.0        0.0       0.0      0.0
 1.13074    1.0        0.0        0.0       0.0      0.0
 1.25257    1.26042    1.0        0.0       0.0      0.0
 0.93774    0.180152   0.10936    1.0       0.0      0.0
 0.259795  -0.390211  -0.14451   -1.36984   1.0      0.0
 0.599616  -0.496896   0.148289   3.78942  -1.52944  1.0,

6x6 Array{Float64,2}:
 0.694002   0.958648   0.656373    0.864156   0.655747   0.108587  
 0.0       -0.491041   0.0358197  -0.149813  -0.443124   0.517778  
 0.0        0.0       -0.728584   -0.466039   0.476894  -0.0847179 
 0.0        0.0        0.0        -0.133753   0.193447   0.00762118
 0.0        0.0        0.0         0.0        0.328977   0.528736  
 0.0        0.0        0.0         0.0        0.0        1.87893   )

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0          0.0  0.0  0.0  0.0  0.0
 0.0          0.0  0.0  0.0  0.0  0.0
 1.11022e-16  0.0  0.0  0.0  0.0  0.0
 0.0          0.0  0.0  0.0  0.0  0.0
 0.0          0.0  0.0  0.0  0.0  0.0
 0.0          0.0  0.0  0.0  0.0  0.0

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.844135
 0.281712
 0.345159
 0.387557
 0.354447
 0.648639

In [7]:
x=A\b

6-element Array{Float64,1}:
  0.0901867
  2.92866  
 -1.1668   
 -0.194855 
 -1.67618  
  0.0679026

In [8]:
y=myL(L,b)

6-element Array{Float64,1}:
  0.844135
 -0.672785
  0.135808
 -0.297671
 -0.51552 
  0.127584

In [9]:
x1=myU(U,y)

6-element Array{Float64,1}:
  0.0901867
  2.92866  
 -1.1668   
 -0.194855 
 -1.67618  
  0.0679026

In [10]:
x-x1

6-element Array{Float64,1}:
 -9.71445e-17
  0.0        
 -6.66134e-16
  7.21645e-16
 -4.44089e-16
  8.32667e-17

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1.

In [11]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [12]:
mylu1(A)

6x6 Array{Float64,2}:
 0.694002   0.958648   0.656373    0.864156   0.655747   0.108587  
 1.13074   -0.491041   0.0358197  -0.149813  -0.443124   0.517778  
 1.25257    1.26042   -0.728584   -0.466039   0.476894  -0.0847179 
 0.93774    0.180152   0.10936    -0.133753   0.193447   0.00762118
 0.259795  -0.390211  -0.14451    -1.36984    0.328977   0.528736  
 0.599616  -0.496896   0.148289    3.78942   -1.52944    1.87893   

In [13]:
L,U

(
6x6 Array{Float64,2}:
 1.0        0.0        0.0        0.0       0.0      0.0
 1.13074    1.0        0.0        0.0       0.0      0.0
 1.25257    1.26042    1.0        0.0       0.0      0.0
 0.93774    0.180152   0.10936    1.0       0.0      0.0
 0.259795  -0.390211  -0.14451   -1.36984   1.0      0.0
 0.599616  -0.496896   0.148289   3.78942  -1.52944  1.0,

6x6 Array{Float64,2}:
 0.694002   0.958648   0.656373    0.864156   0.655747   0.108587  
 0.0       -0.491041   0.0358197  -0.149813  -0.443124   0.517778  
 0.0        0.0       -0.728584   -0.466039   0.476894  -0.0847179 
 0.0        0.0        0.0        -0.133753   0.193447   0.00762118
 0.0        0.0        0.0         0.0        0.328977   0.528736  
 0.0        0.0        0.0         0.0        0.0        1.87893   )

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [14]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.903796    0.738309   0.695912  …  0.0902884  0.818886    0.446701 
 0.0717313   0.600272   0.235524     0.132131   0.261135    0.873703 
 0.909408    0.947099   0.978692     0.453131   0.881104    0.722411 
 0.54517     0.432489   0.328485     0.810108   0.00890598  0.925249 
 0.85106     0.921674   0.139709     0.185145   0.786884    0.476957 
 0.541853    0.980055   0.702073  …  0.797795   0.301344    0.935229 
 0.731343    0.341836   0.563567     0.263423   0.99831     0.80876  
 0.153017    0.400878   0.109396     0.958214   0.232474    0.870347 
 0.0570459   0.195312   0.316343     0.0690949  0.660693    0.262871 
 0.552332    0.209838   0.219945     0.147531   0.150723    0.731427 
 0.520277    0.68459    0.373622  …  0.497      0.32923     0.0992118
 0.551132    0.222228   0.903428     0.283562   0.886816    0.950251 
 0.0791112   0.300856   0.922835     0.386875   0.489645    0.923038 
 ⋮                                ⋱             ⋮               

In [15]:
@time lu(A);

In [16]:
@time mylu1(A);

0.189969 seconds (177.58 k allocations: 14.420 MB, 2.34% gc time)
  

### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [17]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [18]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [19]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.612029  0.696121  0.19648   0.372235
 0.186628  0.224185  0.312353  0.205039
 0.981409  0.164931  0.560429  0.313918
 0.192397  0.762444  0.504759  0.525801                  …  4x4 Array{Float64,2}:
 0.399887  0.50596   0.06536   0.611653
 0.230605  0.730287  0.839388  0.729475
 0.715488  0.318252  0.598433  0.639638
 0.712234  0.367139  0.170803  0.875623                        
 4x4 Array{Float64,2}:
  0.393898   -1.69764   0.0968984   1.00255
 -5.71644   -31.0488    7.48604    13.45   
 -5.97127   -25.8495    7.30293    11.7698 
  1.13856     7.48456  -1.41951    -1.62566     4x4 Array{Float64,2}:
 -0.450881   1.20328   2.03232   0.807206
 -4.55215   18.7946   20.1531   10.0765  
 -4.81435   15.805    15.7394    8.45582 
  0.164645  -4.79444  -4.24579  -3.06917                 
 4x4 Array{Float64,2}:
 -2.75834    -5.61298   2.08025    3.76764
  0.0968047   1.68812  -0.299278   0.5294 
  3.54162    16.0145   -3.30137   -6.50282
  1.38672   

0.387931 seconds (17.26 k allocations: 1.004 GB, 9.62% gc time)


In [20]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [21]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                              …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                                
 4x4 Array{Float64,2}:
  0.0           0.0           5.55112e-17  0.0        
  2.66454e-15   6.66134e-16   2.88658e-15  1.11022e-15
  0.0          -2.22045e-16   1.55431e-15  0.0        
 -5.55112e-17   2.22045e-16  -3.33067e-16  3.33067e-16     4x4 Array{Float64,2}:
 0.0          0.0           2.22045e-16   0.0        
 4.44089e-16  0.0          -1.11022e-15   0.0        
 4.44089e-16  4.44089e-16  -8.88178e-16  -4.44089e-16
 0.0          0.0           4.44089e-16  -2.22045e-16     

In [22]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [23]:
norm(unblock(Res))

1.3880650088705668e-14

Sada probajmo veću dimenziju

In [24]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [25]:
@time mylu2(Ab);

Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se rietci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednsti u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> što u praksi dovoljno spriječava rast elemenata.


In [26]:
A=rand(5,5)
L,U,P=lu(A)

(
5x5 Array{Float64,2}:
 1.0        0.0         0.0       0.0       0.0
 0.203265   1.0         0.0       0.0       0.0
 0.261295   0.957551    1.0       0.0       0.0
 0.211457   0.422875   -0.249327  1.0       0.0
 0.428942  -0.0959707   0.922999  0.110296  1.0,

5x5 Array{Float64,2}:
 0.847692  0.391778  0.982708   0.802882    0.122705
 0.0       0.724498  0.280257  -0.0442058   0.674159
 0.0       0.0       0.353751   0.0837194  -0.266715
 0.0       0.0       0.0        0.0562533   0.574858
 0.0       0.0       0.0        0.0         1.18878 ,

[2,5,1,4,3])

In [27]:
L*U-A[P,:]

0.071067 seconds (8.79 k allocations: 26.526 MB, 5.17% gc time)


5x5 Array{Float64,2}:
  0.0          0.0   0.0          0.0  0.0
  0.0          0.0   0.0          0.0  0.0
 -2.77556e-17  0.0  -1.11022e-16  0.0  0.0
 -2.77556e-17  0.0   2.77556e-17  0.0  0.0
  0.0          0.0   0.0          0.0  0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [28]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [29]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.970802
 0.947262
 0.583406
 0.974863
 0.495018

In [30]:
L,U,Pr,Pc=gecp(A)

(
5x5 Array{Float64,2}:
 1.0        0.0        0.0        0.0       0.0
 0.403938   1.0        0.0        0.0       0.0
 0.387639  -0.112222   1.0        0.0       0.0
 0.710276   0.539328  -0.750881   1.0       0.0
 0.218263   0.277415   0.789361  -0.253873  1.0,

5x5 Array{Float64,2}:
 0.913058  0.28805   0.401305   0.747311  0.430187
 0.0       0.711384  0.22746    0.295997  0.674495
 0.0       0.0       0.512794  -0.231761  0.333971
 0.0       0.0       0.0       -0.399404  0.153552
 0.0       0.0       0.0        0.0       0.243453,

5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0,

5x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0)

In [31]:
Pr*L*U*Pc'-A

5x5 Array{Float64,2}:
 0.0   0.0          0.0  0.0  0.0
 0.0   0.0          0.0  0.0  0.0
 0.0  -1.11022e-16  0.0  0.0  0.0
 0.0   0.0          0.0  0.0  0.0
 0.0   0.0          0.0  0.0  0.0

In [32]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
 0.970802
 0.102874
 0.610088
 0.296488
 0.300524

In [33]:
z=myU(U,y)

5-element Array{Float64,1}:
  0.8996  
 -0.999052
  0.264769
 -0.267751
  1.23442 

In [34]:
x=Pc*z

5-element Array{Float64,1}:
  1.23442 
 -0.999052
 -0.267751
  0.8996  
  0.264769

In [35]:
A*x-b

5-element Array{Float64,1}:
 -1.11022e-16
 -1.11022e-16
  0.0        
 -2.22045e-16
  0.0        

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Porgeska_unatrag_i stabilni_algoritmi](NA04_Pogeska_unatrag_i stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za danai problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$
za neki $\hat x=x+\delta x$.

Želimo ocijeniti 
$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 87, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$
odnosno
$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$
odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$ imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je _uvjetovanost_ (_kondicija_)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [R. Scitovski, Numerička matematika, str. 42][RS04]:




[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, osijek, 2004."

In [36]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [37]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [38]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [39]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [40]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [41]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [42]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 122, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(n^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1) slijedi 
da za relativnu pogreška izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

In [43]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.67457   
 0.568465  
 0.00404135
 0.867243  
 0.494696  
 0.155616  
 0.591977  
 0.5308    
 0.798096  
 0.305927  

In [44]:
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0        1.0         1.0          …  1.0         1.0       1.0        
 0.67457    0.568465    0.00404135      0.5308      0.798096  0.305927   
 0.455044   0.323153    1.63325e-5      0.281748    0.636957  0.0935911  
 0.306959   0.183701    6.60056e-8      0.149552    0.508352  0.028632   
 0.207065   0.104428    2.66752e-10     0.0793821   0.405714  0.00875929 
 0.13968    0.0593635   1.07804e-12  …  0.042136    0.323798  0.0026797  
 0.0942239  0.0337461   4.35674e-15     0.0223658   0.258422  0.000819791
 0.0635606  0.0191835   1.76071e-17     0.0118718   0.206246  0.000250796
 0.042876   0.0109051   7.11567e-20     0.00630152  0.164604  7.67251e-5 
 0.0289229  0.00619918  2.87569e-22     0.00334485  0.13137   2.34722e-5 

In [45]:
b=rand(n)

10-element Array{Float64,1}:
 0.993882 
 0.414292 
 0.852137 
 0.0519895
 0.216377 
 0.994487 
 0.543635 
 0.658461 
 0.933354 
 0.916854 

In [46]:
x=A\b

10-element Array{Float64,1}:
       3.33836e6
       8.90701e7
   -2891.05     
   40326.6      
       1.70399e7
   34699.8      
      -5.24642e7
      -5.65679e7
      -2.4768e5 
 -240730.0      

In [47]:
cond(A)

2.5638805957595735e9

In [48]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
  3.338363566816710975148955811438766764140435919473479216260326930254518450327471e+06
  8.907007912964351751077219761002907068624740435504628016407497974457313306403037e+07
 -2.891046037891099715901809005389843873396782116660560384277966440649619992701596e+03
  4.032658073023228114091579799265556228852233540799552370038666491398393170237661e+04
  1.703992961867523931371723658420525520157733585337082852231250041976319202843816e+07
  3.469977266768251705419927600185071351065282449139070917468003311133281418557857e+04
 -5.246421716879110538664569484407950967877869569745112530262855328608209542869817e+07
 -5.656787938038539282753576237187281037290067592972374013637913399592403591478083e+07
 -2.476800590615323529405209696929777946368560616709724193403424893179490926191726e+05
 -2.407300203753361490162770134580227598357801405732032558530660548514302325943339e+05

In [49]:
norm(xb-x)/norm(xb)

6.536293332258814538491735317715866101813876083404201203436491015903454844835148e-10

### Umjetno loša kondicija

In [50]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)

([-1.0,2.0],50000.00017991671,[8.881784197001252e-16,-4.440892098500626e-16])

## Rezidual

(Prema [G. W. Stewart, Afternotes on Numerical Analysis, str. 128.][Ste96])

Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava

$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

_rezidual_ (ili _ostatak_) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$
pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,
> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$
vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$
Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphi, 1996"

In [51]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [52]:
norm(r)/(norm(A)*norm(x))

0.0